## Building a search engine

Today we will write code to search a small corpus of documents for anything you are interested in. The corpus are the roughly 6,000 ``Featured Articles`` on Wikipedia.

### Exercise 1: Text Preprocessing
To warm up, finish three of the four functions below.

In [8]:
import string
import numpy as np

stopwords = set([
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
    "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers",
    "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves",
    "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are",
    "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does",
    "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until",
    "while", "of", "at", "by", "for", "with", "about", "against", "between", "into",
    "through", "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here",
    "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more",
    "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so",
    "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"
])


def to_lowercase(text):
    '''
    Takes a string, and returns it with every character lowercase
    For example, it should map AABBccDD to aabbccdd
    '''
    return text.lower()

def remove_punctuation(text):
    '''
    Takes a string and returns that string with all punctuation removed.
    '''
    return text.translate(str.maketrans('', '', string.punctuation))


def tokenize(text):
    '''
    Takes a string and returns a list of words or 'tokens' in that string, in order
    Tokens are characters delimited by spaces, for example:
    "The quick brown fox" -> ['The','quick','brown','fox']
    '''
    return list(text.split(' '))

def remove_stop_words(tokenized_text):
    '''
    Removes common words from a list of tokens:
    ['the','quick','brown','fox'] -> ['quick','brown','fox']
    '''
    return list(filter(lambda x: x not in stopwords, tokenized_text))


### Exercise 2: Document Representation
Now we need to write a function that takes a list of documents, called a `Corpus`, and generates a sorted list of its vocabulary. In other words, we need a list of all the words that appear in all of our documents.

Then we need to define a function takes a document and a corpus vocabulary, and generates a word vector for that document. 

In [10]:
def create_vocabulary(documents):
    '''
    Takes a list of documents and returns a sorted list of 
    all the words in any of those documents.
    '''
    vocabulary = set()
    for doc in documents:
        words = remove_stop_words(tokenize(remove_punctuation(to_lowercase(doc))))
        vocabulary.update(words) #add them to the vocabulary
    return sorted(vocabulary) #return the vocabulary

In [5]:


def document_to_vector(document, vocabulary):
    '''
    Takes a specific document and the sorted corpus-wide 
    vocabulary, and returns a vector
    counting how many times each words appears in the specific document.
    '''
    word_count = {}
    for word in vocabulary:
        word_count[word] = 0
    
    words = remove_stop_words(tokenize(remove_punctuation(to_lowercase(document))))
    for word in words: 
        if word in word_count:
            word_count[word] += 1
    return np.array([word_count[word] for word in vocabulary])


In [6]:
test_corpus = [
    'there once was a cat in hungary, who wore a hat',
    'a quick brown fox',
    'jumped over the lazy dog',
    'how now brown cow',
    'once upon a time, there was a big big big big bad wolf',
    'It was raining cats and dogs, before it was sunny'
]

In [11]:
print(create_vocabulary(test_corpus))

['bad', 'big', 'brown', 'cat', 'cats', 'cow', 'dog', 'dogs', 'fox', 'hat', 'hungary', 'jumped', 'lazy', 'quick', 'raining', 'sunny', 'time', 'upon', 'wolf', 'wore']


In [12]:
vocab = create_vocabulary(test_corpus)
document_to_vector(test_corpus[4],vocab)

array([1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0])

### Exercise 3: Building the Document-Term Matrix
Now we want to construct a matrix where each row represents a document and each column represents a term from the vocabulary.

In [14]:
import numpy as np
def create_document_term_matrix(documents, vocabulary):
    '''
    This function loops over all the documents in the corpus, 
    extracts the vocabulary vector of each one, and adds it to
    list. Finally, it transforms this list of vectors into a matrix.
    '''
    document_vectors = []
    for doc in documents:
        doc_vector = document_to_vector(doc, vocabulary)
        document_vectors.append(doc_vector)
    return np.array(document_vectors)


In [15]:
#test:
print(test_corpus)
vocab = create_vocabulary(test_corpus)
dtm=create_document_term_matrix(test_corpus,vocab)
dtm

['there once was a cat in hungary, who wore a hat', 'a quick brown fox', 'jumped over the lazy dog', 'how now brown cow', 'once upon a time, there was a big big big big bad wolf', 'It was raining cats and dogs, before it was sunny']


array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]])

## Exercise 4: Search Query Processing
Now that we have code to transform a corpus of text in a matrix, we want to be able to search the corpus via the matrix. To do so, we also need to write code that transforms a user search query into a vector.

In [31]:
def preprocess_query(query):
    '''
    A function to take a query (a string) and process it in the same way we processed
    the texts in our corpus, getting tokens out in the end
    '''
    query_lower = to_lowercase(query)
    query_no_punct = remove_punctuation(query_lower)
    query_tokens = remove_stop_words(query_no_punct)
    return remove_stop_words(query_tokens)

def query_to_vector(query, vocabulary):
    '''
    A function mapping a query to a vector
    '''
    query_vector = np.zeros(len(vocabulary))
    tokens = preprocess_query(query)
    for token in tokens: 
        if token in vocabulary:
            index = vocabulary.index(token)
            query_vector[index] += 1
    return query_vector


In [17]:
np.zeros(len(vocab))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [18]:
print(vocab)

['bad', 'big', 'brown', 'cat', 'cats', 'cow', 'dog', 'dogs', 'fox', 'hat', 'hungary', 'jumped', 'lazy', 'quick', 'raining', 'sunny', 'time', 'upon', 'wolf', 'wore']


In [19]:
query_to_vector('How many cats are there in Hungary?',vocab)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

### Exercise 5: Ranking Documents
Now we have vectors for documents, and vectors for potential queries about those documents. Given a query we want to know: what is the closest document? Technically: what document vector is closest to the query vector?

One popular measure of the similarity between two vectors is cosine similarity, which uses the idea that the cosine of a small angle is small. This works even in higher dimensions!

![title](cos.png)

Source: https://medium.com/@milana.shxanukova15/cosine-distance-and-cosine-similarity-a5da0e4d9ded


In [28]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0: #if either vector is 0, just return 0
        return 0
    return dot_product / (norm_vec1 * norm_vec2)


In [32]:
def rank_documents(query, document_term_matrix, vocabulary, documents):
    '''
    Given a query, a document-term matrix, a vocabulary, and a set of documents,
    return the documents listed in order of relevance to the query.
    '''
    #note: feel free to take multiple lines to solve any part of this.
    query_vector = query_to_vector(query, vocabulary)
    similarities = [cosine_similarity(doc_vector, query_vector) for doc_vector in document_term_matrix]
    ranked_documents = sorted(zip(similarities, documents), key=lambda x: x[0], reverse=True)
    return ranked_documents


In [33]:
test_corpus

['there once was a cat in hungary, who wore a hat',
 'a quick brown fox',
 'jumped over the lazy dog',
 'how now brown cow',
 'once upon a time, there was a big big big big bad wolf',
 'It was raining cats and dogs, before it was sunny']

In [34]:
rank_documents('who is the biggest brown lazy cat in hungary?',dtm,vocab,test_corpus)

[(0, 'there once was a cat in hungary, who wore a hat'),
 (0, 'a quick brown fox'),
 (0, 'jumped over the lazy dog'),
 (0, 'how now brown cow'),
 (0, 'once upon a time, there was a big big big big bad wolf'),
 (0, 'It was raining cats and dogs, before it was sunny')]

## Putting it all together.
What's missing? Well... real data! I've given you a folder containing the text of 6000+ featured Wikipedia articles. Let's try to make a document term matrix out of this and query it.

Just run this code, no need for coding.

In [35]:
import glob #glob is a library that helps read in lots of files

new_texts=[]
for text_file in glob.glob('data/*.txt')[0:1000]: #glob creates a list of files following a pattern
    #note you could make this run faster by considering only the first 1000 articles in the line above.
    with open(text_file,'r',encoding='utf-8') as f:
        new_texts.append((''.join(f.readlines()))[0:3000])  #cap the file at 3000 lines
print('data read in!')

vocab2=create_vocabulary(new_texts)
print('vocab vector generated!')
        
dtm2=create_document_term_matrix(new_texts,vocab2)
print('dtm generated!')

data read in!
vocab vector generated!
dtm generated!


In [36]:
dtm2.shape

(1000, 38978)

In [38]:
rank_documents('What is the meaning of life?',dtm2,vocab2,new_texts)[0]

(0.06899341558261021,
 'Barbara L (1947–1977) was an American Quarter Horse that raced during the early 1950s and often defeated some of the best racehorses of the time. She earned $32,836 (equivalent to $370,000 in 2023) on the race track in 81 starts and 21 wins, including six wins in stakes races. She set two track records during her racing career. After retiring from racing in 1955, she went on to become a broodmare and had 14 foals, including 11 who earned their Race Register of Merit with the American Quarter Horse Association (AQHA). Her offspring earned more than $200,000 in race money. She died in 1977 and was inducted into the AQHA\'s American Quarter Horse Hall of Fame in 2007.\n\n\n== Early life ==\nBarbara L was foaled in 1947, a bay daughter of a Thoroughbred stallion named Patriotic and a Quarter Horse broodmare named Big Bess. She was registered with the AQHA as number 146,954. Her sire, or father, was a grandson of Man o\' War, while her dam, or mother, descended from 

In [39]:
rank_documents('Is data science going to be a good profession in the future?',dtm2,vocab2,new_texts)[0]

(0.1118033988749895,
 'The banded sugar ant (Camponotus consobrinus), also known as the sugar ant, is a species of ant native to Australia. A member of the genus Camponotus in the subfamily Formicinae, it was described by German entomologist Wilhelm Ferdinand Erichson in 1842. Its common name refers to the ant\'s liking for sugar and sweet food, as well as the distinctive orange-brown band that wraps around its gaster.\nThe ant is polymorphic and relatively large, with two different castes of workers: major workers (also known as soldiers), and minor workers. These two group of workers measure around 5 to 15 millimetres (0.2 to 0.6 in) in length, while the queen ants are even larger. Mainly nocturnal, banded sugar ants prefer a mesic habitat, and are commonly found in forests and woodlands. They also occur in urban areas, where they are considered a household pest. The ant\'s diet includes sweet secretions that are retrieved from aphids and other insects that it tends. This species is 

### Why is this so slow?
There are many reasons why this solution is so slow. Two reasons:
1. This is a lot of data - check how many unique tokens there are in our corpus.
2. The matrix is very big. How many entries do we have? (# of unique tokens x # of texts - we are counting 0s!)

In a future text mining class you'll learn how to solve these kinds of issues!

In [40]:
#number of unique words, size of the matrix, size of the data in the matrix in mb
len(vocab), len(vocab)*len(new_texts),(len(vocab)*len(new_texts)*8)/1_000_000

(20, 20000, 0.16)